In [15]:
import os
import numpy as np
import cv2
from scipy.stats import kurtosis, skew
from scipy import ndimage
import skimage.measure

In [16]:
def generateDataForImages():
    """
    In this function, various techniques for augementing image data, such as random rotations
    and translations, zooming on images will performed to produce a dataset of size ~1600 images.
    RETURN :
        a list of 2-tuples where the first element in each tuple is an image's numpy 2-D array 
        and the second label is the corresponding label, 1 for real note and 0 for fake note
    """
    
    # this will be the function used to resize and grayscale the raw input image

    def resizeGrayCenter(img):
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (600, 300), interpolation = cv2.INTER_AREA)
        img = img - img.mean()
        return img
    
 
    # a list of 2-tuples to hold an image's numpy 2-D array 
    # and the corresponding label, 1 for real and 0 for fake
    trainingData = []
    noOfRows = 300
    noOfCols = 600
    
    # first all real currency notes' images and then all the fake ones'
    # i am skeptical of the consequences :-/
    label = 1
    
    for directory in ['Real Notes 2000/', 'Duplicate Notes 2000/']:
        for filename in os.listdir(directory):
            print (filename)
            img = cv2.imread(directory + filename)
            img = resizeGrayCenter(img)

            # TRANSLATIONS
            # will produce (6 + 6 + 1) x (6 + 6 + 1) = 169 images
            # stride of 5 pixels along both axis along all 4 directions
            for x in range(30, -35, -5):
                for y in range(30, -35, -5):
                    translationMatrix = np.float32([ [1,0,x], [0,1,y] ])
                    imgTrns = cv2.warpAffine(img, translationMatrix, (noOfCols, noOfRows))
                    trainingData.append((np.var(imgTrns),skew(imgTrns,axis=None),kurtosis(imgTrns,axis=None),skimage.measure.shannon_entropy(imgTrns),label))


            # ROTATIONS
            # we produce 41 different angles in the range of -10 to 10
            # with the step being equal to 0.5
            for angle in range(20, -21, -1):
                rotationMatrix = cv2.getRotationMatrix2D((noOfCols/2, noOfRows/2), float(angle)/2, 1)
                imgRotated = cv2.warpAffine(img, rotationMatrix, (noOfCols, noOfRows))
                trainingData.append((np.var(imgRotated),skew(imgRotated,axis=None),kurtosis(imgRotated,axis=None),skimage.measure.shannon_entropy(imgRotated), label))

            # PROJECTIVE TRANSFORMATIONS for ZOOMING IN AND ZOOMING OUT
            # will produce (30 + 30) images for the dataset
            # 1ST ZOOMING IN ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]])
                dstPoints = np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.var(imgZoomed),skew(imgZoomed,axis=None),kurtosis(imgZoomed,axis=None),skimage.measure.shannon_entropy(imgZoomed), label))
            # 2ND ZOOMING OUT ...
            for step in np.arange(0.001, 0.031, 0.001):
                srcPoints = np.float32(np.float32([[0,0], [noOfCols-1,0], [0,noOfRows-1], [noOfCols-1,noOfRows-1]]))
                dstPoints = np.float32([[int(step*(noOfCols-1)),int(step*(noOfRows-1))], [int((1-step)*(noOfCols-1)),int(step*(noOfRows-1))], [int(step*(noOfCols-1)),int((1-step)*(noOfRows-1))], [int((1-step)*(noOfCols-1)), int((1-step)*(noOfRows-1))]]) 
                projective_matrix = cv2.getPerspectiveTransform(srcPoints, dstPoints)
                imgZoomed = cv2.warpPerspective(img, projective_matrix, (noOfCols,noOfRows))
                trainingData.append((np.var(imgZoomed),skew(imgZoomed,axis=None),kurtosis(imgZoomed,axis=None),skimage.measure.shannon_entropy(imgZoomed), label))
        # set label for fake images to come
        label = 0
            
    return trainingData

In [17]:
imagenlabel=generateDataForImages()
len(imagenlabel)


Real_1.jpg
Real_2.jpg
Real_3.jpg
Duplicate_1.jpg
Duplicate_2.jpg
Duplicate_3.jpg


1620

In [18]:
from numpy.random import shuffle
imagenlabel=5*imagenlabel
shuffle(imagenlabel)
imagenlabel

[(1318.7321053977325,
  -1.7881632754044474,
  4.562114309033592,
  6.635228313993791,
  0),
 (1328.7626066033758,
  -1.2925137116608993,
  3.642022999481778,
  15.293972962854468,
  1),
 (1267.8430428495765,
  -1.4176495591668479,
  4.3593644447545365,
  6.679825652690373,
  1),
 (1360.1167578596633,
  -1.674905387510416,
  3.794109035857023,
  14.7267483926787,
  0),
 (1437.5076783809643,
  -1.5712578454065,
  3.327487714382781,
  15.138042590292825,
  0),
 (1305.9336865577893,
  -1.3148681697525557,
  3.7757778886344555,
  6.781645278867964,
  1),
 (1409.060140498168,
  -1.5996478467114805,
  3.4683582918447824,
  6.739014170295273,
  0),
 (1402.221482442443,
  -1.616314048614788,
  3.56680804857157,
  6.653694998632727,
  0),
 (1192.7890553069133,
  -1.8176060013997641,
  4.943014785296664,
  6.440301220785818,
  0),
 (1305.7225829962651,
  -1.6373456477758903,
  3.9132411162106173,
  14.190580918558904,
  0),
 (1256.8607402121156,
  -1.4446025856137217,
  4.5080729970181554,
  6.6

In [19]:
import csv

In [20]:
with open('dataset.csv', 'w') as f: 
    write = csv.writer(f)  
    write.writerows(imagenlabel) 